In [ ]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
# @Title: ArcGIS Online Administration Notebook
#
# @Purpose: this notebook is a sample of how the ArcGIS Python
# API can be used to adminster a portal. Specifically, this
# notebook generates a report of: 
# 1) items in the portal, including those with the word 'test' in the tag or the title
# 2) user status based on last login and activities
# 3) Hosted feature services used in the org
# The reports update a hosted table connected to a dashboard for interactive exploration. 
#
# @Creator: ckwon@esri.com
# @Credits: Building upon work done by Geospatial Center SEs
# @Last Updated: March 2021
#
# @Versions: ArcGIS Python API v1.8.4
#
# @License: See end of notebook
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

# Import Libraries & Connect to the portal

In [2]:
#Import relevant libraries
import io
import csv
import time
import getpass
import requests
import datetime
import pandas as pd
from arcgis.features import FeatureLayer
from arcgis.gis import GIS, Item, User
from arcgis.gis.admin import License, LicenseManager

In [3]:
##REQUIRE USER INPUT##
#Assign ArcGIS Online organization for administration
org = 'https://esriaiddev.maps.arcgis.com/home'
username = 'ckwon_aid'
password = getpass.getpass(prompt='Password:')

Password: ············


In [4]:
#Connect to the GIS environment
gis = GIS(url=org, username=username, password=password)

# Item Administration

In [5]:
#From the REST URL, request all items in the organization. 
#Create a dictionary of the results, and print out the output. 
url = f'{gis.url}/sharing/rest/content/portals/{gis.properties.id}'

params = {
    'f': 'csv',
    'token': gis._portal.con.token
}

#Get a string response from the request and construct a DataFrame
csv_out = requests.get(url, params=params).text
df = pd.read_csv(io.StringIO(str(csv_out)))

#Replace NaN values in the df as presence of NaN results in error when adding features to the feature table
df_filled = df.fillna('None')
df_filled

,id,title,owner,fullname,size,created,modified,url
0,c13d470a1c024318ad221d75412649b4,SIDW_Forum_Sponsors,ssawaya_aid,Salim Sawaya,11105,2013-06-02 16:57:52.0,2021-03-22 19:41:53.0,None
1,d3bf9513eb6148e99df2436563fe3167,SIDW_Forum_Sponsors,ssawaya_aid,Salim Sawaya,57344,2013-06-02 16:57:55.0,2013-06-06 06:07:16.0,http://services.arcgis.com/LG9Yn2oFqZi5PnO5/ar...
2,8bfad945962f4d40b0ae7ce7f74293e3,SID-W 2013 Sponsors HQs,ssawaya_aid,Salim Sawaya,2781,2013-06-02 17:09:41.0,2013-06-06 17:02:16.0,None
3,b6f3ab78228f45578427ee1f055b0995,SID-W 2013 Sponsors,ssawaya_aid,Salim Sawaya,488,2013-06-02 17:11:19.0,2013-06-02 21:19:29.0,http://esriaiddev.maps.arcgis.com/apps/SocialM...
4,cb2cbb56c69746b0b5b29753c6bf404b,AgMarketFinder,ssawaya_aid,Salim Sawaya,0,2013-06-03 01:41:47.0,2013-06-03 05:45:51.0,http://marketfinder.info/
...,...,...,...,...,...,...,...,...
5398,76fc35d2fac04119a1a8a684f216299c,Environmental Impact Public Comment,ajenkins_EsriAidDev,Adam Jenkins,3721,2021-03-26 15:04:48.0,2021-03-26 15:09:18.0,https://EsriAidDev.maps.arcgis.com/apps/Crowds...
5399,dba564a23ac84b90bc4cfd2fc287a39f,Global Stream Flow Web Map-AJ,ajenkins_EsriAidDev,Adam Jenkins,3086,2021-03-26 19:20:39.0,2021-03-26 19:20:43.0,None
5400,2edf8faf5cc84269b6aad2d930f4d0ab,Stream,ajenkins_EsriAidDev,Adam Jenkins,670,2021-03-26 19:23:23.0,2021-03-26 19:24:53.0,https://esriaiddev.maps.arcgis.com/apps/instan...
5401,c40a3e5e9db3427b9804c836335c4a54,Test Example,ajenkins_EsriAidDev,Adam Jenkins,991,2021-03-26 19:26:35.0,2021-03-26 19:27:02.0,https://esriaiddev.maps.arcgis.com/apps/instan...


In [6]:
#Iterate through each record, add new item attributes, and determine if 'test' exists in the title & the tag.

f_item = []
attributeMap = ['id','title','owner','fullname','created','modified','url','size']

for index, row in df_filled.iterrows():
    #Create an empty dictionary for each item
    f = {"attributes":{}}
    
    #Write the row of each record to the dictionary
    for field in attributeMap:
        f['attributes'][field] = row[field]
    
    #Then convert size in bytes to megabytes, format to six decimal places & update
    size_mb = int(row['size'])/1000000
    f['attributes']['size_mb'] = "{:.6f}".format(size_mb)
    
    #Get the item and include additional fields
    #Documentation on available fields: https://developers.arcgis.com/rest/users-groups-and-items/common-parameters.htm#ESRI_SECTION1_1FFBA7FE775B4BDA8D97524A6B9F7C98
    item = gis.content.get(itemid=row['id'])
    f['attributes']['access'] = item['access']
    f['attributes']['type'] = item['type']
    f['attributes']['snippet'] = item['snippet']
    f['attributes']['categories'] = str(item['categories']).strip('[]') #Convert to string & remove the []. A list cannot be inserted into a string field in the hosted table
    f['attributes']['numComments'] = item['numComments']
    f['attributes']['numRatings'] = item['numRatings']
    f['attributes']['numViews'] = item['numViews']
    f['attributes']['scoreCompleteness'] = item['scoreCompleteness']
    f['attributes']['tags'] = str(item['tags']).strip('[]') #Convert to string & remove the []. A list cannot be inserted into a string field in the hosted table
    
    #For each item, check if the title or tag contains the word 'test'
    #First check if the item has a title. If it does not, indicate the item. 
    #Then check if the title has the word 'test,' followed by the tag
    #Assign a variable for title, tag, and id to simplify logic below
    item_title = f['attributes']['title']
    item_tag = f['attributes']['tags']
    item_id = f['attributes']['id']
    
    if isinstance(item_title, float): 
        print(f'Item ID:{item_id} does not have a valid title')
    else: 
        if 'test' in item_title:
            f['attributes']['tag_status'] = 'yes'
        elif item_tag != '':
            if 'test' in item_tag:
                f['attributes']['tag_status'] = 'yes'
            else:
                f['attributes']['tag_status'] = 'no'            
        else:
            f['attributes']['tag_status'] = 'no'            
    
    #Append the record to the list 
    f_item.append(f)

#Display the first 5 items as output
f_item[:1]

[{'attributes': {'id': 'c13d470a1c024318ad221d75412649b4',
   'title': 'SIDW_Forum_Sponsors',
   'owner': 'ssawaya_aid',
   'fullname': 'Salim Sawaya',
   'created': '2013-06-02 16:57:52.0',
   'modified': '2021-03-22 19:41:53.0',
   'url': 'None',
   'size': 11105,
   'size_mb': '0.011105',
   'access': 'private',
   'type': 'CSV',
   'snippet': None,
   'categories': '',
   'numComments': 0,
   'numRatings': 0,
   'numViews': 1,
   'scoreCompleteness': 33,
   'tags': '',
   'tag_status': 'no'}}]

In [7]:
#Count the number of items where tag_status = 'yes' for reporting purposes  
count_yes = 0
count_no = 0

for record in f_item:
    tag_status = record.get('attributes',{}).get('tag_status')
    if tag_status == 'yes':
        count_yes += 1
    else:
        count_no += 1

count_total = count_yes + count_no
        
print(f'{count_total} items were checked...')
print(f'{count_yes} items have the word "test" in the title or the tag.')

5403 items were checked...
267 items have the word "test" in the title or the tag.


In [8]:
##REQUIRE USER INPUT##
#Select the Hosted Table for update

#If this is the first time running the notebook, create the hosted table with the following CSV. 
#https://esriis-my.sharepoint.com/:x:/g/personal/cal10660_esri_com/EYi8huq-F25AipgAONwVK5IB5NOOIFop9WaHXteq9TGJSw?e=ZCdFNa

#Make sure to set the field types correctly.
#String: id, title, owner, fullname, url, access, type, snippet, categories, tags, tag_status
#Integer: numComments, numRatings, numViews, scoreCompleteness
#Double: size, size_mb
#Date: created, modified

item_id = '4602725d51234a7ba6887b5b491d3b0e'
dest_fl = gis.content.get(item_id).tables[0]
dest_fl

<Table url:"https://services.arcgis.com/LG9Yn2oFqZi5PnO5/arcgis/rest/services/item_details/FeatureServer/0">

In [9]:
#Update the Hosted Table with the item details
dest_fl.delete_features(where="1=1")
result = dest_fl.edit_features(adds=f_item)
print('success...')

success...


# User Administration

In [10]:
#Search for users in the organization 
search = 1000 #Make sure this value exceeds the no. of users in the org
users_all = gis.users.search(max_users=search)
users_count = len(users_all)
print(f'{users_count} users found...')

95 users found...


In [11]:
#Iterate through each user, determine various properties, and create a list of dictionary
#Refer to properties of User Object: https://developers.arcgis.com/rest/users-groups-and-items/user.htm
f_user = []
attributeMap = ['username','id','fullName','availableCredits','assignedCredits','preferredView', 'email','lastLogin','mfaEnabled','access',
                'orgId','role','privileges','userLicenseTypeId','disabled', 'region','thumbnail','created','modified','groups','provider']

for user in users_all:
    f = {"attributes":{}}
    for field in attributeMap: 
        f['attributes'][field] = user[field]
        
        #Format privileges and add to the dictionary
        if field == 'privileges':
            privilege = user[field]
            privilege_ls = []
            i = 0
            while i < len(user[field]): #Since privileges assigned varies with each user, find the length of the list.
                privilege_str = user[field][i] #Take the i-th item in the list 
                privilege_name = privilege_str.rsplit(':',1)[1] #Format from [portal]:[usertype]:[privilege] to [privilege]
                privilege_ls.append(privilege_name) #Append the formatted string to a list
                i = i + 1
            f['attributes']['user_privileges'] = str(privilege_ls).strip('[]').replace('\'','') #Remove the brackets and quotation mark to insert the string to the dictionary.
            del f['attributes'][field]                                                          #Privilege is a SQL keyword so change to user_privileges. Delete Privileges key from the dict.
        
        #Count the number of groups
        if field == 'groups':
            group_count = len(user[field])
            f['attributes'][field] = group_count
            
    #Count the number of items owned by each user
    item_count = len(user.items(max_items = 1000)) #Max_item is defaulted to 100
    f['attributes']['item'] = item_count
    
    #Determine My Esri access status
    myesri_access = user.esri_access
    f['attributes']['myEsri'] = myesri_access
    
    #Append the record to the list 
    f_user.append(f)

#Display the first 5 items as output
f_user[:1]

[{'attributes': {'username': 'AGiron_aid',
   'id': 'a64c4d9e27704dc2929b83a68947b436',
   'fullName': 'Amanda Giron',
   'availableCredits': 1000.0,
   'assignedCredits': 1000.0,
   'preferredView': None,
   'email': 'AGiron@esri.com',
   'lastLogin': 1607967224000,
   'mfaEnabled': False,
   'access': 'org',
   'orgId': 'LG9Yn2oFqZi5PnO5',
   'role': 'org_publisher',
   'user_privileges': 'edit, bulkPublishFromDataStores, publishDynamicImagery, publishFeatures, publishScenes, publishServerServices, publishTiledImagery, publishTiles, registerDataStores, categorizeItems, createGroup, createItem, joinGroup, joinNonOrgGroup, shareGroupToOrg, shareGroupToPublic, shareToGroup, shareToOrg, shareToPublic, viewOrgGroups, viewOrgItems, viewOrgUsers, geoanalytics, demographics, elevation, featurereport, geocode, stored, temporary, geoenrichment, networkanalysis, closestfacility, locationallocation, optimizedrouting, origindestinationcostmatrix, routing, servicearea, vehiclerouting, spatialanaly

In [12]:
##REQUIRE USER INPUT##
#Select the Hosted Table for update

#If this is the first time running the notebook, create the hosted table with the following CSV. 
#https://esriis-my.sharepoint.com/:x:/g/personal/cal10660_esri_com/EUPUqn1jvk9BptlGbTPmOEYBVuJOvsAUHjy5tsBXuLGimw?e=adOeGU

#Make sure to set the field types correctly.
#String: username, id, fullName, preferredView, email, mfaEnabled, access, orgId, role, privileges, userLicenseTypeId, disabled, region, thumbnail, provider, myEsri
#Integer: storageUsage, storageQuota, groups, item
#Double: availableCredits, assignedCredits
#Date: lastLogin, created, modified

item_id = '93c7b07deea146e58b42ca154f439a5e'
dest_fl = gis.content.get(item_id).tables[0]
dest_fl

<Table url:"https://services.arcgis.com/LG9Yn2oFqZi5PnO5/arcgis/rest/services/user_details/FeatureServer/0">

In [13]:
#Update the Hosted Table with the item details
dest_fl.delete_features(where="1=1")
result = dest_fl.edit_features(adds=f_user)
print('success...')

success...


# Hosted Feature Service Tracking

In [14]:
# @Purpose: Hosted feature service is the most versatile and frequently used type of layer in ArcGIS Online.
# But with the versatility comes cost storing the layers and they are generally the leading source of credit consumption.
# This notebook is meant to be run once a day to keep a running history of feature service usage in the organization 
# in a format that is easy to digest and explore, enabling the administrator to maintain oversight of usage. 

In [15]:
#Take the f_item generated from the previous steps and create a new list with type = Feature Service
fl_lst = []
date_today = datetime.datetime.today().strftime('%Y/%m/%d') #Date when the code is run 

for items in f_item:
    if items['attributes']['type'] == 'Feature Service':
        items['attributes']['date_updated'] = date_today
        items['attributes']['credit_cost'] = float(items['attributes']['size_mb']) * .24 #Convert storage to credit cost
        items['attributes']['remaining_credits'] =  gis.admin.credits.credits #Get the remaining credits in the organization
        fl_lst.append(items)

#Add the user's last login information to the item
for fl in fl_lst:
    item_owner = fl['attributes']['owner']
    
    for user in f_user:
        username = user['attributes']['username'] 
        
        if item_owner == username:
            fl['attributes']['last_login'] = user['attributes']['lastLogin'] 

#Summarize the results
fl_count = len(fl_lst)
print(f'{fl_count} feature layers found in the organization...')
fl_lst[:1]

1246 feature layers found in the organization...


[{'attributes': {'id': 'd3bf9513eb6148e99df2436563fe3167',
   'title': 'SIDW_Forum_Sponsors',
   'owner': 'ssawaya_aid',
   'fullname': 'Salim Sawaya',
   'created': '2013-06-02 16:57:55.0',
   'modified': '2013-06-06 06:07:16.0',
   'url': 'http://services.arcgis.com/LG9Yn2oFqZi5PnO5/arcgis/rest/services/SIDW_Forum_Sponsors/FeatureServer',
   'size': 57344,
   'size_mb': '0.057344',
   'access': 'public',
   'type': 'Feature Service',
   'snippet': '',
   'categories': '',
   'numComments': 0,
   'numRatings': 0,
   'numViews': 170,
   'scoreCompleteness': 33,
   'tags': "'SIDW'",
   'tag_status': 'no',
   'date_updated': '2021/03/29',
   'credit_cost': 0.01376256,
   'remaining_credits': 50571.32,
   'last_login': 1611754636000}}]

In [16]:
##REQUIRE USER INPUT##
#Select the Hosted Table for update

#If this is the first time running the notebook, create the hosted table with the following CSV. 
#https://esriis-my.sharepoint.com/:x:/g/personal/cal10660_esri_com/EeTMhaHSsK9EnjJwYrtnJJ4BNOx-qLU-dcJu_DC5i6rquQ?e=I9ZEDY

#Make sure to set the field types correctly.
#String: id, title, owner, fullname, url, access, type, snippet, categories, tags, tag_status
#Integer: numComments, numRatings, numViews, scoreCompleteness
#Double: size, size_mb
#Date: created, modified, date_updated

item_id = '4f9ea13534304f799a8378af30b63c72'
dest_fl = gis.content.get(item_id).tables[0]
dest_fl

<Table url:"https://services.arcgis.com/LG9Yn2oFqZi5PnO5/arcgis/rest/services/fl_details/FeatureServer/0">

In [17]:
#Update the Hosted Table with the item details
dest_fl.delete_features(where=f'date_updated = \'{date_today}\'') #If the notebook is run more than once a day, delete any previous records from the same day. 
result = dest_fl.edit_features(adds=fl_lst)
print('success...')

success...


# License Administration

In [18]:
# licenses = gis.admin.license.all()

# f_license = []

# for license in licenses:
#     f = {"attributes":{}}
#     try:
#         report = license.report
#         #print(report)
#         for index, row in report.iterrows():
#             licensename = license.properties.listing.title
#             #print(licensename)
#             #print(report.shape[0])
#             if report.shape[0] > 1:
#                 f['attributes']['Entitlement'] = row['Entitlement']
#                 f['attributes']['Total'] = row['Total']
#                 f['attributes']['Assigned'] = row['Assigned']
#                 f['attributes']['Remaining'] = row['Remaining']
#                 f_license.append(f)
#     except:
#         continue
        
# f_license

# Licensing

In [19]:
# '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
#    Copyright © 2021 Esri
#
#    All rights reserved under the copyright laws of the United States 
#    and applicable international laws, treaties, and conventions.
#    You may freely redistribute and use this sample code, with or 
#    without modification, provided you include the original copyright 
#    notice and use restrictions.
#
#    Disclaimer: THE SAMPLE CODE IS PROVIDED "AS IS" AND ANY EXPRESS 
#    OR IMPLIED WARRANTIES, INCLUDING THE IMPLIED WARRANTIES OF 
#    MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE 
#    DISCLAIMED. IN NO EVENT SHALL ESRI OR CONTRIBUTORS BE LIABLE FOR
#    ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL 
#    DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS 
#    OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) 
#    SUSTAINED BY YOU OR A THIRD PARTY, HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, 
#    WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT ARISING IN ANY WAY OUT OF THE USE 
#    OF THIS SAMPLE CODE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
#
#    For additional information, contact:
#    Esri
#    Attn: Contracts and Legal Services Department
#    380 New York Street
#    Redlands, California, 92373-8100
#    USA
#    email: contracts@esri.com
# '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''